## Exploring the biosphere matrix and characterisation factors

#### Massimo Pizzol, 2024


In [1]:
# Note: must be environment bw2
import brightway2 as bw
import pandas as pd
import numpy as np

In [16]:
# any project with a version of ecoinvent (3.9.1 in this case) 
# and the Biosphere database installed
bw.projects.set_current("advlca23") 

In [3]:
# list of methods
#bw.methods

In [26]:
# a climate change LCIA method
mymethod = ('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)')

In [22]:
# Select an activity with carbon dioxide as exchange
act = bw.Database('ecoinvent 3.9 conseq').get('8660ffb81a69adf8a51014a4a77f2305')

In [23]:
# find if there is a carbon dioxide exchange
[exc for exc in list(act.exchanges()) if exc['name'] == 'Carbon dioxide, fossil'] 
# yes there is one

[Exchange: 0.55731 kilogram 'Carbon dioxide, fossil' (kilogram, None, ('air',)) to 'manganese(III) oxide production' (kilogram, RoW, None)>]

In [25]:
# calculate LCA score 
lca = bw.LCA({act: 1} , mymethod)
lca.lci()
lca.lcia()
lca.score

2.263494134704986

## Find the code for the "carbon dioxide" exchange(s)

In [27]:
for i in bw.Database('biosphere3').search('carbon dioxide fossil'):
    print(i)
    print(i['code'])

'Carbon dioxide, fossil' (kilogram, None, ('air', 'low population density, long-term'))
e259263c-d1f1-449f-bb9b-73c6d0a32a00
'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks'))
aa7cac3a-3625-41d4-bc54-33e2cf11ec46
'Carbon dioxide, fossil' (kilogram, None, ('air',))
349b29d1-3e58-4c66-98b9-9d1a076efd2e
'Carbon dioxide, fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere'))
16eeda8a-1ea2-408e-ab37-2648495058dd
'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))
f9749677-9c9f-4678-ab55-c607dfdc2cb9
'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks'))
d6235194-e4e6-4548-bfa3-ac095131aef4
'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'low population density, long-term'))
28e1e2d6-97ad-4dfd-932a-9edad36dcab9
'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere'))
4e1f0bb0-2703-4303-bf86-972d810612cf
'Carbon dioxide, non-f

## Find the coordinates of the exchange in the intervention (biosphere) matrix

First look at the sizes of these matrices.

In [11]:
lca.technosphere_matrix

<18856x18856 sparse matrix of type '<class 'numpy.float64'>'
	with 229695 stored elements in Compressed Sparse Row format>

In [12]:
lca.biosphere_matrix

<2419x18856 sparse matrix of type '<class 'numpy.float64'>'
	with 317013 stored elements in Compressed Sparse Row format>

In [13]:
col = lca.activity_dict[(act['database'], act['code'])] # find column index of A matrix for the activit
print('index/coordinate of the activity column:', col)
row = lca.biosphere_dict[('biosphere3', '349b29d1-3e58-4c66-98b9-9d1a076efd2e')] # find row index of B matrix for the exchange
print('index/coordinate of the exchange row:', row)

index/coordinate of the activity column: 13812
index/coordinate of the exchange row: 689


In [14]:
# this is the value of the exchange in the activity 
print('value/amount of the exchange:', lca.biosphere_matrix[row,col])

value/amount of the exchange: 0.5573099851608276


## Find characterisation factors for specific exchanges...

If you want to find the characterisation factor for a known exchange

In [29]:
# a square matrix of characterisation facors for the method chosen
lca.characterization_matrix

<2419x2419 sparse matrix of type '<class 'numpy.float64'>'
	with 67 stored elements in Compressed Sparse Row format>

In [30]:
# This is the value of the characterisaiton factor in the method used
print(lca.characterization_matrix[row,row])

1.0


## ...and exchanges that match specific charachterisation factors

If you find a characterisation factor and want to find which exchange it refers to 

In [31]:
# This is the name of the excahnge starting from the index, in the biosphere ematrix
# Note I know it's the biosphere matrix because [2] indicates that matrix
lca.reverse_dict()[2][row]

('biosphere3', '349b29d1-3e58-4c66-98b9-9d1a076efd2e')

In [159]:
# This would give something very different 
# because [1] means we are looking into the technosphere matrix
lca.reverse_dict()[1][row]

('ecoinvent 3.9 conseq', '2e8900639b17f306093810c317d98184')

In [32]:
#just the same but hard coded
lca.characterization_matrix[82,82]

1.0

## All charactersiation factors

The entire list of charachterisation factors. We see the indexes but we don't see which exchanges they refer to (that is why the code just two cells above is useful)

In [160]:
print(lca.characterization_matrix)

  (82, 82)	1.0
  (86, 86)	4.062399864196777
  (87, 87)	2.490999937057495
  (96, 96)	16.401809692382812
  (164, 164)	1301.2701416015625
  (165, 165)	137.56021118164062
  (166, 166)	0.8982621431350708
  (167, 167)	11123.494140625
  (274, 274)	2.354546546936035
  (275, 275)	6291.62939453125
  (276, 276)	1764.6295166015625
  (277, 277)	8.915194511413574
  (278, 278)	10239.234375
  (279, 279)	147.66224670410156
  (280, 280)	29.700000762939453
  (281, 281)	28.5
  (282, 282)	1728.470703125
  (283, 283)	6625.7802734375
  (284, 284)	4662.93701171875
  (285, 285)	12397.603515625
  (312, 312)	16069.998046875
  (460, 460)	23506.8203125
  (686, 686)	1.0
  (687, 687)	1.0
  (688, 688)	1.0
  :	:
  (829, 829)	29.700000762939453
  (830, 830)	29.700000762939453
  (831, 831)	29.700000762939453
  (832, 832)	12.1832275390625
  (833, 833)	12.1832275390625
  (834, 834)	1728.470703125
  (835, 835)	6625.7802734375
  (836, 836)	12397.603515625
  (947, 947)	23506.8203125
  (948, 948)	23506.8203125
  (1706, 1706)	

In [34]:
# to get these indices as a list one can use
np.argwhere(lca.characterization_matrix)[0:10].tolist() # gives the indices as a list
# found here https://stackoverflow.com/questions/40313886/extracting-items-from-sparse-matrix

[[82, 82],
 [96, 96],
 [164, 164],
 [165, 165],
 [167, 167],
 [274, 274],
 [275, 275],
 [276, 276],
 [277, 277],
 [278, 278]]

In [35]:
# matching characterisation factors with exchanges
for i in np.argwhere(lca.characterization_matrix)[0:5]:
    myactid = lca.reverse_dict()[2][i[0]]
    print(myactid)
    print(bw.Database('biosphere3').get(myactid[1]))
    print(lca.characterization_matrix[i[0],i[1]])
    print('---')

('biosphere3', 'f9749677-9c9f-4678-ab55-c607dfdc2cb9')
'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))
1.0
---
('biosphere3', '4cadbc9f-0472-4bb9-a942-df5b90e37878')
'Chloroform' (kilogram, None, ('air', 'urban air close to ground'))
16.0
---
('biosphere3', 'fa0c2bee-8dd9-4f8a-8489-b1f3b43de958')
'Ethane, 1,1,1,2-tetrafluoro-, HFC-134a' (kilogram, None, ('air', 'urban air close to ground'))
1300.0
---
('biosphere3', '393d0862-04cc-45ae-8d08-9dc2a4461b35')
'Ethane, 1,1-difluoro-, HFC-152a' (kilogram, None, ('air', 'urban air close to ground'))
138.0
---
('biosphere3', 'df5dd437-2e12-4af6-8f7a-9c8224857dc5')
'Ethane, hexafluoro-, HFC-116' (kilogram, None, ('air', 'urban air close to ground'))
11100.0
---


In [36]:
#same but formatted nice
full_names = []
ids = []
cfs = []

for i in np.argwhere(lca.characterization_matrix)[0:10]:
    myactid = lca.reverse_dict()[2][i[0]]
    ids.append(myactid)
    full_names.append(bw.Database('biosphere3').get(myactid[1])['name'])
    cfs.append(lca.characterization_matrix[i[0],i[1]])

In [37]:
pd.DataFrame({'full_name' : full_names, 
              'id' : ids,  
              'cf' : cfs})

,full_name,id,cf
0,"Carbon dioxide, fossil","(biosphere3, f9749677-9c9f-4678-ab55-c607dfdc2...",1.0
1,Chloroform,"(biosphere3, 4cadbc9f-0472-4bb9-a942-df5b90e37...",16.0
2,"Ethane, 1,1,1,2-tetrafluoro-, HFC-134a","(biosphere3, fa0c2bee-8dd9-4f8a-8489-b1f3b43de...",1300.0
3,"Ethane, 1,1-difluoro-, HFC-152a","(biosphere3, 393d0862-04cc-45ae-8d08-9dc2a4461...",138.0
4,"Ethane, hexafluoro-, HFC-116","(biosphere3, df5dd437-2e12-4af6-8f7a-9c8224857...",11100.0
5,"Methane, bromo-, Halon 1001","(biosphere3, 4bf1c333-419b-4278-ac14-247c8bf56...",2.0
6,"Methane, bromotrifluoro-, Halon 1301","(biosphere3, 18ddd879-5496-4419-8ab7-e0d3864d9...",6290.0
7,"Methane, chlorodifluoro-, HCFC-22","(biosphere3, 8ca6c84b-7f56-47e6-8d17-4f2dbc49a...",1760.0
8,"Methane, dichloro-, HCC-30","(biosphere3, 2716807d-0266-48ff-bb7a-6e6c5fbfd...",9.0
9,"Methane, dichlorodifluoro-, CFC-12","(biosphere3, 31a3e44c-4098-461e-aba2-f79540e05...",10200.0


## Now we change method and see what happens

In [39]:
#bw.methods

In [40]:
anothermethod =  ('CML v4.8 2016', 'human toxicity', 'human toxicity (HTP inf)')

lca = bw.LCA({act: 1} , anothermethod)
lca.lci()
lca.lcia()
lca.score

1.6372387965226802

In [41]:
lca.characterization_matrix # this is the same dimension 
# but different number of elemnts

<2419x2419 sparse matrix of type '<class 'numpy.float64'>'
	with 539 stored elements in Compressed Sparse Row format>

In [42]:
print(lca.characterization_matrix[row,row]) # Different CF of course.

0.0


In [43]:
# finally the entire characterisation matrix
print(lca.characterization_matrix)

  (3, 3)	46.952484130859375
  (13, 13)	572399.625
  (14, 14)	21.6158390045166
  (21, 21)	56.906005859375
  (25, 25)	506.5604553222656
  (26, 26)	4669.0390625
  (30, 30)	0.10000000149011612
  (37, 37)	6707.54248046875
  (41, 41)	21.29570198059082
  (42, 42)	38.961204528808594
  (44, 44)	756.4636840820312
  (46, 46)	0.42841529846191406
  (48, 48)	15.08444881439209
  (51, 51)	1899.9091796875
  (52, 52)	0.9732104539871216
  (53, 53)	3157103.0
  (54, 54)	409.47552490234375
  (56, 56)	226630.84375
  (58, 58)	28.594619750976562
  (69, 69)	2224.3916015625
  (72, 72)	145040.546875
  (73, 73)	22.890777587890625
  (76, 76)	0.09748458117246628
  (77, 77)	20.96550941467285
  (78, 78)	141.18975830078125
  :	:
  (2309, 2309)	3.3102262020111084
  (2325, 2325)	44.34622573852539
  (2326, 2326)	0.13927416503429413
  (2327, 2327)	0.0006488700164481997
  (2328, 2328)	0.10979720950126648
  (2329, 2329)	0.6930022239685059
  (2330, 2330)	0.5543568730354309
  (2333, 2333)	0.8759815692901611
  (2336, 2336)	3.26

In [44]:
for exc in list(act.exchanges()):
    if exc['input'] == ('biosphere3', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'):
        print("the exchange is: ", exc)
        print("the flow value is:", exc['amount'])
        print("the cf value is:",lca.characterization_matrix[row,row])
        print("the characterized impact is:", exc['amount'] * lca.characterization_matrix[row,row])

the exchange is:  Exchange: 0.55731 kilogram 'Carbon dioxide, fossil' (kilogram, None, ('air',)) to 'manganese(III) oxide production' (kilogram, RoW, None)>
the flow value is: 0.55731
the cf value is: 0.0
the characterized impact is: 0.0
